In [1]:
import pandas as pd
import numpy as np
from datetime import date
import holidays
from prophet import Prophet

import datetime
from datetime import timedelta
from statsmodels.tsa.seasonal import seasonal_decompose
from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor


from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score, GridSearchCV

tscv = TimeSeriesSplit(n_splits=5)
state = np.random.RandomState(12345)

C:\Users\14488\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('sp_sales_task\\train_cluster_2.csv')
df = df[(df['st_id'] == 7) & (df['pr_sku_id'] == 1780)]

In [3]:
df = df.rename(columns = {'pr_sales_in_rub' : 'y'})

In [4]:
# df['ds'] = pd.to_datetime(df['date'])

In [5]:
df = df.drop(['date', 'month_year'], axis=1)

In [6]:
df.dropna (how='all')

,Unnamed: 0,st_id,pr_sku_id,pr_sales_type_id,pr_sales_in_units,y,price_units,price_pred,price_units_log,st_city_id,...,pr_group_id_group,year,month,week,day,holiday,liquidity,top_st_revenue,cluster_kmean,cluster_st
9661,9661,7,1780,0,2,249,124.500000,0.135335,4.824306,2,...,1,2023,5,3,4,0,1,0,11.0,3.0
27239,27239,7,1780,0,1,160,160.000000,0.367879,5.075174,2,...,1,2023,7,0,3,0,1,0,11.0,3.0
38397,38397,7,1780,1,1,117,117.000000,0.367879,4.762174,2,...,1,2023,6,5,17,1,1,0,11.0,3.0
40630,40630,7,1780,0,1,204,204.000000,0.367879,5.318120,2,...,1,2023,5,1,2,1,1,0,11.0,3.0
50712,50712,7,1780,0,2,322,161.000000,0.135335,5.081404,2,...,1,2023,6,4,23,0,1,0,11.0,3.0
75052,75052,7,1780,1,1,197,197.000000,0.367879,5.283204,2,...,1,2023,6,6,25,1,1,0,11.0,3.0
78708,78708,7,1780,0,3,432,144.000000,0.049787,4.969813,2,...,1,2023,1,6,8,0,1,0,11.0,3.0
112220,112220,7,1780,0,1,117,117.000000,0.367879,4.762174,2,...,1,2023,5,5,13,1,1,0,11.0,3.0
142552,142552,7,1780,0,3,508,169.333333,0.049787,5.131869,2,...,1,2023,6,1,20,0,1,0,11.0,3.0
168268,168268,7,1780,0,2,275,137.500000,0.135335,4.923624,2,...,1,2023,1,0,16,0,1,0,11.0,3.0


In [7]:
# Создадим пустой Датафрем для записи результатов
data_analys = pd.DataFrame(columns = ['LinearRegression', 'RandomForestRegressor', 
                          'CatBoostRegressor'], 
                           index=['WAPE'])

In [8]:
# Создадим тестовую выборку размером 10% от исходных данных.
# Разделим тренировочную выборку для валидации моделей

train, test = train_test_split(df, shuffle=False, random_state=state, test_size=0.1)

print('Размер тренировочной выборки:', train.shape)
print('Размер тестовой выборки:', test.shape)

Размер тренировочной выборки: (53, 33)
Размер тестовой выборки: (6, 33)


In [9]:
X_train = train.drop('y', axis=1)
y_train = train['y']
X_test = test.drop('y', axis=1)
y_test = test['y']

In [10]:
model = LinearRegression().fit(X_train, y_train)

# Проверим выбранную модель на тестовых данных

test['predict_lr'] = model.predict(X_test)
# Вычисляем метрику
data_analys.loc['WAPE', 'LinearRegression'] = 100 * (test['y'] - test['predict_lr']).abs().sum() / test['y'].sum()

In [11]:
# Случайный лес на дефолтных параметрах
model_forest = RandomForestRegressor(random_state=state) 
model_forest.fit(X_train, y_train) 
test['predict_forest'] = model_forest.predict(X_test)
# Вычисляем метрику
data_analys.loc['WAPE', 'RandomForestRegressor'] = 100 * (test['y'] - test['predict_forest']).abs().sum() / test['y'].sum() 


In [12]:
# Проверим модель на дефолтных параметрах

model_cbr = CatBoostRegressor(
   loss_function='RMSE'
).fit(X_train, y_train, verbose=False)
test['predict_cat'] = model_cbr.predict(X_test)
# Вычисляем метрику
data_analys.loc['WAPE', 'CatBoostRegressor'] = 100 * (test['y'] - test['predict_cat']).abs().sum() / test['y'].sum() 


In [13]:
data_analys

,LinearRegression,RandomForestRegressor,CatBoostRegressor
WAPE,16.653664,16.768248,20.669724


In [14]:
test[['y', 'predict_lr', 'predict_forest', 'predict_cat']]

,y,predict_lr,predict_forest,predict_cat
749558,256,281.360222,256.52,261.942183
758502,117,120.558705,109.59,159.520895
770465,125,116.196928,125.22,160.136730
784163,222,253.014373,254.61,259.769270
789113,75,50.491480,98.85,128.505218
814817,301,390.279264,420.17,352.665876
